In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1

     |████████████████████████████████| 3.8MB 3.5MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 3.9MB 37.1MB/s 
     |████████████████████████████████| 450kB 38.6MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [ ]:
import torch
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import tokenize
import logging
from tqdm import tqdm
from tqdm import trange
from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)

def prepare_ctrl_input(temperature, _, tokenizer, prompt_text):
    if temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(xlm_language, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if xlm_language in available_languages:
            language = xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(padding_text, _, tokenizer, prompt_text):
    prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


def prepare_transfoxl_input(padding_text, _, tokenizer, prompt_text):
    prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}

MAX_LENGTH = int(10000)

def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def initialize_model(model_type, length):

  MODEL_CLASSES = {
      "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
      "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
      "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
      "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
      "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
      "xlm": (XLMWithLMHeadModel, XLMTokenizer),
  }
  try:
      model_class, tokenizer_class = MODEL_CLASSES[model_type]
  except KeyError:
      raise KeyError("the model {} you specified is not supported.")
  tokenizer = tokenizer_class.from_pretrained(model_type)
  model = model_class.from_pretrained(model_type)
  model.to(device)
  length = adjust_length_to_model(length, max_sequence_length=model.config.max_position_embeddings)
  return model, tokenizer, length

In [ ]:
model_type = "ctrl" #PUT NAME OF NEEDED MODEL
length = 200 #MODEL LENGTH
prompt_text = "Links Java vs Python"
repetition_penalty = 1.2
temperature = 0.2
stop_token = None
num_return_sequences = 5
# Initialize the model and tokenizer
model, tokenizer, length =  initialize_model(model_type, length)

In [ ]:
def generate_text_from_condition(model, tokenizer, length, prompt_text, repetition_penalty, temperature, num_return_sequences, model_name, stop_token = None):
   # Initialize the model and tokenizer
    # prompt_text = args.prompt if args.prompt else input("Model prompt >>> ")

    # Different models need different input formatting and/or extra arguments
  requires_preprocessing = model_type in PREPROCESSING_FUNCTIONS.keys()
  if requires_preprocessing:
      prepare_input = PREPROCESSING_FUNCTIONS.get(model_type)
      preprocessed_prompt_text = prepare_input(temperature, model, tokenizer, prompt_text)
      encoded_prompt = tokenizer.encode(
          preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", add_space_before_punct_symbol=True
      )
  else:
      encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(device)

  if encoded_prompt.size()[-1] == 0:
      input_ids = None
  else:
      input_ids = encoded_prompt

  output_sequences = model.generate(
      input_ids=input_ids,
      max_length=length + len(encoded_prompt[0]),
      temperature=temperature,
      # top_k=args.k,
      # top_p=args.p,
      repetition_penalty=repetition_penalty,
      do_sample=True,
      num_return_sequences=num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
      output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      generated_sequence = generated_sequence.tolist()

      # Decode text
      text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      # Remove all text after the stop token
      text = text[: text.find(stop_token) if stop_token else None]

      # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
      total_sequence = (
          prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
      )
      if model_name == 'ctrl':
        total_sequence = total_sequence.replace(prompt_text + " \n \n ", "")
        total_sequence = " ".join(tokenize.sent_tokenize(total_sequence)[:-1])
      else:
        total_sequence = total_sequence.replace(prompt_text + "\n", "")
        total_sequence = " ".join(tokenize.sent_tokenize(total_sequence)[:-1])
      generated_sequences.append(total_sequence)
  return generated_sequences

gen = generate_text_from_condition(model, tokenizer, length, "Links What is better: Java or Python?", repetition_penalty, temperature, num_return_sequences, 'gpt-2', stop_token="<|endoftext|>")
gen

NameError: ignored

In [ ]:
df = pd.read_csv("100_short.csv")
df = df[["object_a", 'object_b']]
df = df.reindex(df.columns.tolist() + ['CTRL_gen_0','CTRL_gen_1', "CTRL_gen_2", "CTRL_gen_3", "CTRL_gen_4", 'GPT_2_gen_0','GPT_2_gen_1', "GPT_2_gen_2", "GPT_2_gen_3", "GPT_2_gen_4"], axis=1)

In [ ]:
question = "Links What is better: {} or {}?"
for i in trange(df.shape[0]):
  obj_a, obj_b = df.iloc[i, 0:2].tolist()
  prompt = question.format(obj_a, obj_b)
  generated_sequences = generate_text_from_condition(model, tokenizer, length, prompt, repetition_penalty, temperature, num_return_sequences, stop_token=None)
  for j in range(num_return_sequences):
    row_ind = df.index[i]
    df.loc[row_ind, 'CTRL_gen_' + str(j)] = generated_sequences[j]

df

,object_a,object_b,CTRL_gen_0,CTRL_gen_1,CTRL_gen_2,CTRL_gen_3,CTRL_gen_4,GPT_2_gen_0,GPT_2_gen_1,GPT_2_gen_2,GPT_2_gen_3,GPT_2_gen_4
0,JavaScript,Perl,I think it's a matter of personal preference. ...,I think that the answer to this question depen...,The answer to this question depends on what yo...,I think that the answer to this question depen...,I think the answer to this question depends on...,NaN,NaN,NaN,NaN,NaN
1,Windows 8,Windows 7,I think the answer to that question depends on...,The answer to that question depends on what yo...,I think the answer to this question depends on...,I think the answer to this question depends on...,I think the answer to that question depends on...,NaN,NaN,NaN,NaN,NaN
2,Scala,Java,I think that the best way to answer this quest...,I think that the answer to this question depen...,I think that the answer to this question depen...,I think the answer to this question depends on...,"I am not a big fan of Scala, but it has some n...",NaN,NaN,NaN,NaN,NaN
3,Swift,Objective-C,I think that the best way to answer this quest...,I think the answer to this question depends on...,I think that the best way to answer this quest...,I think that the answer to this question depen...,I think that the answer to this question depen...,NaN,NaN,NaN,NaN,NaN
4,PHP,Java,I am not sure which one to choose. There are m...,I think that the answer to this question depen...,I have been using both for a long time and the...,I am not a programmer and have no idea what th...,I think that the main reason why people choose...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CNN,Fox,The answer to this question depends on what yo...,The point of this exercise was to see how much...,I think the answer to this question depends on...,The answer to that question depends on what yo...,The answer to that question depends on what yo...,NaN,NaN,NaN,NaN,NaN
96,rat,mouse,I have a friend who has two rats. He feeds the...,The mouse has a much higher chance of being ea...,The answer to this question depends on the typ...,The mouse has a better chance of surviving and...,The answer to this question depends on the typ...,NaN,NaN,NaN,NaN,NaN
97,princeton,harvard,I am a student at the University of Washington...,I think the answer to this question depends on...,I have a friend who has been in the same progr...,I am a student at the University of California...,I am a high school senior and my parents are v...,NaN,NaN,NaN,NaN,NaN
98,iphone,laptop,I have a Macbook Air and it has been great for...,I have a Sony Vaio laptop and it has been work...,I have a dell laptop with windows 7 and it wor...,I have a MacBook Air and an iPad Air. The Surf...,Links What is better: iphone or laptop? I have...,NaN,NaN,NaN,NaN,NaN


In [ ]:
model_type = "gpt2" #PUT NAME OF NEEDED MODEL
length = 200 #MODEL LENGTH
prompt_text = "Links Java vs Python"
repetition_penalty = 1.2
temperature = 0.2
stop_token = None
num_return_sequences = 5
# Initialize the model and tokenizer
model, tokenizer, length =  initialize_model(model_type, length)

In [ ]:
model, tokenizer, length =  initialize_model(model_type, length)
gen = generate_text_from_condition(model, tokenizer, length, "What is better: Java or Python?", repetition_penalty, temperature, num_return_sequences, stop_token=None)
gen

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['What is better: Java or Python? I think it\'s a good question. I\'m not sure if you can answer that one, but there are some things we should be doing to make our code more readable and less verbose (and maybe even faster). We need to have the ability for developers who want their projects written in C++/JavaScript languages to write programs using these tools as well. This means writing tests with them instead of just passing an array through those functions on top; this will help us avoid unnecessary boilerplate when building your own test suites! It also helps keep all other dependencies from being added into production builds so they don\'t get stuck at runtime which makes testing easier once again :) The goal here isn´t simply making my application run fast - rather what does "better" mean? Well...it really depends how much time has passed since then!',
 'What is better: Java or Python? The answer to this question depends on what you are trying and how much time it takes for the 

In [ ]:
question = "What is better: {} or {}?"
for i in trange(df.shape[0]):
  obj_a, obj_b = df.iloc[i, 0:2].tolist()
  prompt = question.format(obj_a, obj_b)
  generated_sequences = generate_text_from_condition(model, tokenizer, length, prompt, repetition_penalty, temperature, num_return_sequences, model_name='gpt-2', stop_token="<|endoftext|>")
  for j in range(num_return_sequences):
    row_ind = df.index[i]
    df.loc[row_ind, 'GPT_2_gen_' + str(j)] = generated_sequences[j]

df



  0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|          | 1/100 [00:16<27:14, 16.51s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  2%|▏         | 2/100 [00:32<26:38, 16.32s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  3%|▎         | 3/100 [00:47<25:46, 15.94s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  4%|▍         | 4/100 [01:02<25:11, 15.74s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  5%|▌         | 5/100 [01:15<23:40, 14.95s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  6%|▌         | 6/100 [01:26<21:24, 13.67s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  7%|▋         | 7/100 [01:40<21:07, 13.63s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  8%|▊

,object_a,object_b,CTRL_gen_0,CTRL_gen_1,CTRL_gen_2,CTRL_gen_3,CTRL_gen_4,GPT_2_gen_0,GPT_2_gen_1,GPT_2_gen_2,GPT_2_gen_3,GPT_2_gen_4
0,JavaScript,Perl,I think it's a matter of personal preference. ...,I think that the answer to this question depen...,The answer to this question depends on what yo...,I think that the answer to this question depen...,I think the answer to this question depends on...,The first thing I want to do with this article...,The answer to this question depends on what yo...,I'm not sure what this means. I think it's tha...,The best way to learn about programming langua...,This question has been asked many times before...
1,Windows 8,Windows 7,I think the answer to that question depends on...,The answer to that question depends on what yo...,I think the answer to this question depends on...,I think the answer to this question depends on...,I think the answer to that question depends on...,The answer to that question depends on your sp...,I've been using the latest version of Microsof...,The first thing I would say to anyone who has ...,The answer to this question depends on your sp...,The answer to this question depends on your pe...
2,Scala,Java,I think that the best way to answer this quest...,I think that the answer to this question depen...,I think that the answer to this question depen...,I think the answer to this question depends on...,"I am not a big fan of Scala, but it has some n...",The answer to this question depends on what yo...,The best way to learn about the language and i...,The answer to this question depends on what yo...,I'm not sure what I'd call it. It's a very sim...,"The first thing to note about this article, an..."
3,Swift,Objective-C,I think that the best way to answer this quest...,I think the answer to this question depends on...,I think that the best way to answer this quest...,I think that the answer to this question depen...,I think that the answer to this question depen...,I'm not sure. I've been using it for a while n...,The answer to this question has been a long ti...,The first thing you should know about the diff...,I'm not sure what to make of this. I think it'...,"The first question I asked myself was, ""How do..."
4,PHP,Java,I am not sure which one to choose. There are m...,I think that the answer to this question depen...,I have been using both for a long time and the...,I am not a programmer and have no idea what th...,I think that the main reason why people choose...,"In the past, I've been a bit of an advocate fo...",,,It's not a question of whether you should use ...,I'm not sure if I can say for certain that the...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CNN,Fox,The answer to this question depends on what yo...,The point of this exercise was to see how much...,I think the answer to this question depends on...,The answer to that question depends on what yo...,The answer to that question depends on what yo...,The answer to that question has been a resound...,The answer to this question depends on how you...,The answer to this question has been a resound...,The answer to this question depends on how you...,"The answer to that question was a resounding ""..."
96,rat,mouse,I have a friend who has two rats. He feeds the...,The mouse has a much higher chance of being ea...,The answer to this question depends on the typ...,The mouse has a better chance of surviving and...,The answer to this question depends on the typ...,The first thing you need to know about the dif...,The answer to this question has been a long ti...,The answer to this question has been a long ti...,The answer to this question has been a long ti...,The answer to this question depends on what yo...
97,princeton,harvard,I am a student at the University of Washington...,I think the answer to this question depends on...,I have a friend who has been in the same progr...,I am a student at the University of California...,I am a high school senior and my parents are v...,The answer to this qu

In [ ]:
df.to_csv("ctrl_gpt.csv")

In [ ]:
d=[]
while(1):
  d.append('1')

KeyboardInterrupt: ignored